In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv=CountVectorizer(stop_words='english',max_features=5000)

In [ ]:
vectors=cv.fit_transform(df['corpus']).toarray()

In [ ]:
from nltk.stem import PorterStemmer

In [ ]:
ps=PorterStemmer()

In [ ]:
def stez(txt):
  l=[]
  for i in txt.split():
    l.append(ps.stem(i))
  return " ".join(l)

In [ ]:
df['corpus']=df['corpus'].apply(stez)

<ipython-input-36-fe7d7a815fa1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['corpus']=df['corpus'].apply(stez)


In [ ]:
vectors=cv.fit_transform(df['corpus']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
similarity.shape

(719, 719)

In [ ]:
list(enumerate(similarity[0]))

[(0, 1.0000000000000002),
 (1, 0.23910503106562464),
 (2, 0.3609941025577121),
 (3, 0.33415943951558474),
 (4, 0.2725326958376928),
 (5, 0.2435993828823451),
 (6, 0.3410955201203233),
 (7, 0.35384615384615387),
 (8, 0.4196811634276874),
 (9, 0.292606782383782),
 (10, 0.358304288091451),
 (11, 0.18752289237539818),
 (12, 0.2864459496157732),
 (13, 0.32573117099222815),
 (14, 0.3669444886153501),
 (15, 0.2170607855311148),
 (16, 0.21787329011536413),
 (17, 0.3130107162734597),
 (18, 0.279078152825719),
 (19, 0.3410781186194002),
 (20, 0.19272482233188631),
 (21, 0.34201772954183957),
 (22, 0.1754116038614058),
 (23, 0.3262589561144743),
 (24, 0.3309112606779028),
 (25, 0.26481167575013703),
 (26, 0.3309112606779028),
 (27, 0.4246176958362201),
 (28, 0.2134163974591771),
 (29, 0.23035022137995856),
 (30, 0.39871588753692655),
 (31, 0.4211597863173308),
 (32, 0.27074557691828405),
 (33, 0.3609941025577121),
 (34, 0.5112313234371975),
 (35, 0.2344964302087667),
 (36, 0.5478938010789937),
 (

In [ ]:
def rec(mobile):
    if mobile not in df['price_group'].values:
        print("Mobile not found in the DataFrame.")
        return []

    index = df[df['price_group'] == mobile].index[0]

    distance = sorted(list(enumerate(similarity[index])), key=lambda x: x[1], reverse=True)

    recommended_names = set()
    recommendations = []

    for i in range(1, len(distance)):
        recommended_index = distance[i][0]

        if recommended_index >= len(df):
            print(f"Index {recommended_index} is out of bounds for the DataFrame.")
            continue

        recommended_mobile = df.iloc[recommended_index]
        mobile_name = recommended_mobile.get('name', 'Unknown')

        if mobile_name not in recommended_names:
            recommendations.append(mobile_name)
            recommended_names.add(mobile_name)

            if len(recommendations) == 6:
                break


    return recommendations




In [ ]:
rec('under 20000')

['SAMSUNG Galaxy M32 ',
 'Redmi 10 Prime ',
 'REDMI Note 10 lite ',
 'OPPO F21 Pro ',
 'REDMI Note 11T 5G ',
 'OnePlus Nord CE 2 5G ']

In [ ]:
import pickle as pk

In [ ]:
pk.dump(df,open('df.pkl','wb'))
pk.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
!pip install streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import pickle
import pandas as pd
import numpy as np

similarity=pickle.load(open('similarity.pkl','rb'))
df=pickle.load(open('df.pkl','rb'))

st.title('Mobile Recommender System')

def rec(mobile):
    if mobile not in df['price_group'].values:
        print("Mobile not found in the DataFrame.")
        return []

    index = df[df['price_group'] == mobile].index[0]

    distance = sorted(list(enumerate(similarity[index])), key=lambda x: x[1], reverse=True)

    recommended_names = set()
    recommendations = []

    for i in range(1, len(distance)):
        recommended_index = distance[i][0]

        if recommended_index >= len(df):
            print(f"Index {recommended_index} is out of bounds for the DataFrame.")
            continue

        recommended_mobile = df.iloc[recommended_index]
        mobile_name = recommended_mobile.get('name', 'Unknown')

        if mobile_name not in recommended_names:
            recommendations.append(mobile_name)
            recommended_names.add(mobile_name)

            if len(recommendations) == 6:
                break


    return recommendations

price_list=(df['price_group'].unique())
price_list.sort()
selected_mobilr=st.selectbox("Select the price below",price_list,index=None)
rec_mobiles=rec(selected_mobilr)
col0,col1,col2,col3,col4 = st.columns(5)
if st.button('Recommend'):
  with col0:
    st.text(rec_mobiles[0])
    #st.image(rec_img[0])
  with col1:
    st.text(rec_mobiles[1])
    #st.image(rec_img[1])
  with col2:
    st.text(rec_mobiles[2])
    #st.image(rec_img[2])
  with col3:
    st.text(rec_mobiles[3])
    #st.image(rec_img[3])
  with col4:
    st.text(rec_mobiles[4])
    #st.image(rec_img[4])

Writing app.py


In [ ]:
!npm install --g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 4s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹npm notice
npm notice New major version of npm available! 10.8.2 -> 11.0.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.0.0
npm notice To update run: npm install -g npm@11.0.0
npm notice
⠹

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙

⠹⠸⠼⠴your url is: https://shaky-shoes-post.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.57.65.202:8501

Mobile not found in the DataFrame.
Mobile not found in the DataFrame.
Mobile not found in the DataFrame.
2024-12-29 15:03:42.395 Uncaught app execution
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/exec_code.py", line 88, in exec_func_with_error_handling
    result = func()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 579, in code_to_exec
    exec(code, module.__dict__)
  File "/content/app.py", line 50, in <module>
    st.text(rec_mobiles[0])
IndexError: list index out of range
Mobile not found in the DataFrame.
2024-12-29 15:03:43.667 Uncaught app execution
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dis